+ Estável

In [36]:
import sqlite3
import pandas as pd
import json
import os
import re
from typing import TypedDict, List, Optional, Any
from datetime import datetime

# Bibliotecas de IA e Grafo
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from rapidfuzz import process, fuzz

# --- CONFIGURAÇÃO ---
DB_PATH = 'db.sqlite3'
LOG_FILE = "historico_conversas_pandas.md"

# --- CLASSE DE DADOS (PANDAS PURO) ---
class ImobiliariaDataEngine:
    def __init__(self):
        # MODO MOCK
        data = {
            'id': [101, 102, 103, 104, 105],
            'titulo': ['Casa Premium', 'Apto Compacto', 'Cobertura Luxo', 'Sítio Verde', 'Casa Vila'],
            'descricao': [
                'Linda residência com piso de porcelanato, área gourmet e garagem.', 
                'Apartamento simples, ideal para estudantes. Sem vaga.', 
                'Vista total, possui jacuzzi e churrasqueira.', 
                'Muito verde, pomar e piscina.', 
                'Casa antiga reformada.'
            ],
            'bairro': ['Benfica', 'Centro', 'São Mateus', 'Aeroporto', 'Benfica'],
            'especificacao': ['3 quartos', '1 quarto', '4 quartos', '5 quartos', '2 quartos'],
            'preco_aluguel': [2500, 800, 5000, 3000, 1200],
            'preco_condominio': [0, 200, 1000, 0, 0],
            'preco_iptu': [100, 50, 500, 200, 80],
            'aceita_pets': [1, 0, 1, 1, 1]
        }
        self.df = pd.DataFrame(data)
        
        # Engenharia de Atributos
        self.df['custo_total'] = self.df['preco_aluguel'] + self.df['preco_condominio'] + self.df['preco_iptu']
        
        # --- NOVO: Coluna legível para a IA ler sobre Pets ---
        self.df['status_pets'] = self.df['aceita_pets'].map({1: 'Aceita Pets', 0: 'Não Aceita'})
        
        # Cria coluna de busca textual
        self.df['texto_busca'] = (
            self.df['titulo'] + " " + 
            self.df['descricao'] + " " + 
            self.df['bairro'] + " " + 
            self.df['especificacao']
        ).fillna('').str.lower()
        
        self.bairros_validos = self.df['bairro'].unique().tolist()
        print(f">>> [INIT] Banco carregado com {len(self.df)} imóveis.")

    def fuzzy_match_bairro(self, bairro_input):
        if not bairro_input: return None
        match = process.extractOne(bairro_input, self.bairros_validos, scorer=fuzz.WRatio)
        if match and match[1] > 80:
            return match[0]
        return None

# --- DEFINIÇÃO DO ESTADO ---
class AgentState(TypedDict):
    input: str                  
    chat_history: List[str]     
    filtros_interpretados: dict 
    df_atual: Any               
    resposta_final: str        

# --- NÓS DO SISTEMA ---
class SistemaImobiliario:
    def __init__(self):
        self.engine = ImobiliariaDataEngine()
        self.llm_json = ChatOllama(model="gemma2:9b", temperature=0, format="json")
        self.llm_text = ChatOllama(model="gemma2:9b", temperature=0.2)
        
        if not os.path.exists(LOG_FILE):
            with open(LOG_FILE, "w", encoding="utf-8") as f:
                f.write("# Histórico de Conversas (Engine Pandas)\n\n")

    def _log(self, user, ai, debug):
        timestamp = datetime.now().strftime("%H:%M:%S")
        with open(LOG_FILE, "a", encoding="utf-8") as f:
            f.write(f"## {timestamp}\n**User:** {user}\n\n**AI:** {ai}\n\n> *Debug:* `{debug}`\n\n---\n")

    # 1. NÓ INTERPRETADOR
    def node_interpretador(self, state: AgentState):
        """Traduz linguagem natural para parâmetros de filtro."""
        print(f"   🧠 [Interpretador] Lendo: '{state['input']}'")
        
        prompt = f"""
        Você é um parser de intenção.
        Analise a frase e extraia um JSON.
        
        INPUT: "{state['input']}"
        CONTEXTO RECENTE: {state['chat_history'][-2:] if state['chat_history'] else 'Nenhum'}
        
        REGRAS:
        1. "keywords": Lista de características FÍSICAS (ex: "piscina", "varanda").
           - Converta para SINGULAR (ex: "vagas" -> "vaga").
           - IGNORE palavras genéricas ("imóvel", "casa", "apto", "lugar", "opção", "quero").
        2. "bairro": Nome do bairro ou null.
        3. "pets": true se mencionar animais/pets, senão null.
        4. "refinamento": true se disser "desses", "da lista", "o mais barato", "entre eles", "esse tem", "quais".
           - Se mudar de bairro/assunto, é false.
        5. "ordenacao": "menor_preco" (se disser "barato"), "maior_preco" ou null.
        
        JSON OUTPUT APENAS:
        """
        response = self.llm_json.invoke(prompt)
        try:
            filtros = json.loads(response.content)
        except:
            filtros = {}
        return {"filtros_interpretados": filtros}

    # 2. NÓ BUSCADOR
    def node_buscador(self, state: AgentState):
        filtros = state['filtros_interpretados']
        
        # Stop Words (Limpeza extra de segurança)
        STOP_WORDS = ['imovel', 'imóvel', 'casa', 'apartamento', 'apto', 'lugar', 'opcao', 'opção', 'quero']
        raw_keywords = filtros.get('keywords', [])
        clean_keywords = [k for k in raw_keywords if k.lower() not in STOP_WORDS]
        filtros['keywords'] = clean_keywords # Atualiza para o log
        
        print(f"   🐼 [Buscador Pandas] Aplicando: {filtros}")
        
        # Lógica de Memória
        usar_memoria = False
        if filtros.get('refinamento') and state.get('df_atual') is not None:
            if not state['df_atual'].empty:
                usar_memoria = True

        if usar_memoria:
            df = state['df_atual'].copy()
            print("      -> Refinando busca anterior...")
        else:
            df = self.engine.df.copy()
            print("      -> Iniciando busca completa...")

        # 1. Bairro
        if filtros.get('bairro'):
            bairro_real = self.engine.fuzzy_match_bairro(filtros['bairro'])
            if bairro_real:
                df = df[df['bairro'] == bairro_real]
                print(f"      -> Bairro corrigido: {bairro_real}")

        # 2. Keywords (Regex)
        if clean_keywords:
            regex_pattern = '|'.join([re.escape(k.lower()) for k in clean_keywords])
            if regex_pattern:
                df = df[df['texto_busca'].str.contains(regex_pattern, regex=True)]
                print(f"      -> Regex aplicado: '{regex_pattern}'")

        # 3. Pets
        if filtros.get('pets'):
            df = df[df['aceita_pets'] == 1]

        # 4. Ordenação
        if filtros.get('ordenacao') == 'menor_preco':
            df = df.sort_values(by='custo_total', ascending=True)
        elif filtros.get('ordenacao') == 'maior_preco':
            df = df.sort_values(by='custo_total', ascending=False)
        
        return {"df_atual": df}

    # 3. NÓ REDATOR (CORRIGIDO PARA VER A COLUNA DE PETS)
    def node_redator(self, state: AgentState):
        df = state['df_atual']
        
        if df.empty:
            res = "Não encontrei imóveis com essas características exatas."
        else:
            top_df = df.head(3)
            
            # --- CRUCIAL: Adicionamos 'status_pets' na visualização ---
            colunas_visiveis = ['titulo', 'bairro', 'custo_total', 'status_pets', 'descricao']
            tabela_md = top_df[colunas_visiveis].to_markdown(index=False)
            
            prompt = f"""
            Você é um assistente imobiliário.
            
            DADOS ENCONTRADOS:
            {tabela_md}

            PERGUNTA DO USUÁRIO: "{state['input']}"

            INSTRUÇÕES:
            1. Use a coluna 'status_pets' para responder sobre animais.
            2. Se perguntou "qual o mais barato", é o primeiro da lista.
            3. Responda de forma natural.
            """
            res = self.llm_text.invoke(prompt).content

        return {"resposta_final": res}

    def build_graph(self):
        workflow = StateGraph(AgentState)
        workflow.add_node("interpretador", self.node_interpretador)
        workflow.add_node("buscador", self.node_buscador)
        workflow.add_node("redator", self.node_redator)
        
        workflow.set_entry_point("interpretador")
        workflow.add_edge("interpretador", "buscador")
        workflow.add_edge("buscador", "redator")
        workflow.add_edge("redator", END)
        return workflow.compile()

# --- EXECUÇÃO: SIMULAÇÃO DE CONVERSA LONGA ---
if __name__ == "__main__":
    app = SistemaImobiliario()
    grafo = app.build_graph()
    
    roteiro_conversa = [
        "Olá, estou procurando um imóvel no bairro Benfica.",
        "Quais desses aceitam animais de estimação?",
        "E qual é o mais barato entre eles?",
        "Esse imóvel tem garagem? Verifica na descrição por favor.",
        "Entendi. Mudei de ideia, me mostre apartamentos no Centro.",
        "Tem algum com porteiro ou segurança?"
    ]

    print("\n" + "="*60)
    print(" 🎭 SIMULAÇÃO DE CONVERSA CONTÍNUA")
    print("="*60)

    memoria_df = None
    historico = []

    for i, pergunta in enumerate(roteiro_conversa, 1):
        print(f"\n👤 USUÁRIO (Turno {i}): {pergunta}")
        
        inputs = {
            "input": pergunta,
            "chat_history": historico,
            "df_atual": memoria_df,
            "filtros_interpretados": {},
            "resposta_final": ""
        }
        
        resultado = grafo.invoke(inputs)
        
        texto = resultado['resposta_final']
        memoria_df = resultado['df_atual']
        filtros = resultado['filtros_interpretados']
        
        print(f"🤖 AGENTE:\n{texto}")
        
        historico.append(f"User: {pergunta}")
        historico.append(f"AI: {texto}")
        app._log(pergunta, texto, filtros)

    print(f"\n✅ Teste finalizado. Logs salvos em: {LOG_FILE}")

>>> [INIT] Banco carregado com 5 imóveis.

 🎭 SIMULAÇÃO DE CONVERSA CONTÍNUA

👤 USUÁRIO (Turno 1): Olá, estou procurando um imóvel no bairro Benfica.
   🧠 [Interpretador] Lendo: 'Olá, estou procurando um imóvel no bairro Benfica.'


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


   🐼 [Buscador Pandas] Aplicando: {'keywords': [], 'bairro': 'Benfica', 'pets': None, 'refinamento': False, 'ordenacao': None}
      -> Iniciando busca completa...
      -> Bairro corrigido: Benfica


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🤖 AGENTE:
Olá!  Temos duas opções no bairro Benfica:

* **Casa Vila:** Uma casa antiga reformada, por R$1280. Aceita pets! 🐶🐱
* **Casa Premium:** Uma residência mais moderna com piso de porcelanato, área gourmet e garagem, por R$2600. Também aceita animais de estimação.  

Qual delas te interessa mais? 😊  




👤 USUÁRIO (Turno 2): Quais desses aceitam animais de estimação?
   🧠 [Interpretador] Lendo: 'Quais desses aceitam animais de estimação?'


...

🤖 AGENTE:
Sim, essa casa tem garagem! A descrição diz que é uma "Linda residência com piso de porcelanato, área gourmet e garagem".  


Gostaria de saber mais sobre alguma outra característica desse imóvel? 


👤 USUÁRIO (Turno 5): Entendi. Mudei de ideia, me mostre apartamentos no Centro.
   🧠 [Interpretador] Lendo: 'Entendi. Mudei de ideia, me mostre apartamentos no Centro.'


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


   🐼 [Buscador Pandas] Aplicando: {'keywords': ['apartamentos'], 'bairro': 'Centro', 'pets': None, 'refinamento': False, 'ordenacao': None}
      -> Iniciando busca completa...
      -> Bairro corrigido: Centro
      -> Regex aplicado: 'apartamentos'
🤖 AGENTE:
Não encontrei imóveis com essas características exatas.

👤 USUÁRIO (Turno 6): Tem algum com porteiro ou segurança?
   🧠 [Interpretador] Lendo: 'Tem algum com porteiro ou segurança?'


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


   🐼 [Buscador Pandas] Aplicando: {'keywords': ['porteiro', 'segurança'], 'bairro': None, 'pets': None, 'refinamento': False, 'ordenacao': None}
      -> Iniciando busca completa...
      -> Regex aplicado: 'porteiro|segurança'
🤖 AGENTE:
Não encontrei imóveis com essas características exatas.

✅ Teste finalizado. Logs salvos em: historico_conversas_pandas.md


em teste